In [113]:
import requests
from bs4 import BeautifulSoup
import time
import csv

names = []
troops  = []
cleaned_data = []
clash_royal = []
csv_file_path = "clash_royal_data.csv"

fieldnames = [
    "name",
    "Rating", 
    "Usage",
    "increase_in_usage",
    "Win",
    "increase_in_win",
    "CWR",
]

def scrape_website(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        a_tags = soup.find_all('a')
        for a_tag in a_tags[1:]:
            img_tag = a_tag.find('img')
            if img_tag:
                try:
                    troop_name = img_tag['alt']
                    names.append(troop_name)
                except:
                    pass
        cards = soup.find_all('div', class_='card_grid__cards_container isotope_grid')
        for card in cards:
            troops.append(card.text)
    else:
        print("Failed to fetch page:", response.status_code)

def wait_crawl_delay():
    time.sleep(2) 
   
def main():
    base_url = 'https://royaleapi.com/cards/popular?time=14d&mode=grid&cat=GC&sort=rating'
    scrape_website(base_url)
    lines = troops[0].split('\n')
    cleaned_data = [item for item in lines if item.strip() != '']
    j = 0
    for i in range(0, len(cleaned_data), 13):
        clash_royal.append(
            {
                "name": names[j],
                cleaned_data[i+2]: cleaned_data[i+1],
                cleaned_data[i+4]: cleaned_data[i+3],
                "increase_in_usage": cleaned_data[i+5] + cleaned_data[i+6],
                cleaned_data[i+8]: cleaned_data[i+7],
                "increase_in_win": cleaned_data[i+9] + cleaned_data[i+10],
                cleaned_data[i+12]: cleaned_data[i+11],
            }
        )
        j += 1
        with open(csv_file_path, "w", newline="") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for obj in clash_royal:
                writer.writerow(
                    {
                        "name": obj["name"],
                        "Rating": obj["Rating "],
                        "Usage": obj["Usage "],
                        "increase_in_usage": obj["increase_in_usage"],
                        "Win": obj["Win "],
                        "increase_in_win": obj["increase_in_win"],
                        "CWR": obj["CWR "],
                    }
                )

if __name__ == "__main__":
    main()
